In [1]:
! pip install pandas
! pip install openpyxl

In [2]:
import pandas as pd

In [3]:
from numpy import *


class MatchRating:
    original_rating_name = "Original"
    avg_player_name = "AVG"
    max_iterations = 1000

    def __init__(self, data_sheet_file, rating_scale_base, rating_scale_divisor, rating_base, tie_in_weight):

        self.rating_scale_base = rating_scale_base
        self.rating_scale_divisor = rating_scale_divisor
        self.ranking_boost = rating_base
        self.tie_in_weight = tie_in_weight
        
        self.players: pd.DataFrame = pd.read_excel(data_sheet_file, sheet_name="Players", index_col="ShortName")
        self.matches: pd.DataFrame = pd.read_excel(data_sheet_file, sheet_name="Matches")
        self.matches["IsTieInMatch"] = False
        
        match_columns = self.matches.columns.to_list()
        self.winner_columns = [column for column in match_columns if column.startswith("Winner")]
        self.loser_columns = [column for column in match_columns if column.startswith("Loser")]

        print('Number of matches loaded:' + str(len(self.matches)))

        # --- Add tie-in matches ---
        cols = self.matches.columns
        first_winner = cols[cols.str.startswith("Winner")].values[0]
        first_loser = cols[cols.str.startswith("Loser")].values[0]
        
        tie_in_rows = []
        for player in self.players.index:
            tie_in_rows.append({first_winner: player, first_loser: MatchRating.avg_player_name, "K": self.tie_in_weight})
            tie_in_rows.append({first_winner: MatchRating.avg_player_name, first_loser: player, "K": self.tie_in_weight})

        tie_in_matches = pd.DataFrame(tie_in_rows)
        tie_in_matches["IsTieInMatch"] = True
        self.matches = pd.concat([self.matches, tie_in_matches], ignore_index=True)

        self.__player_ratings: pd.DataFrame = pd.DataFrame(columns=["Rating"], index=self.players.index)
        self.__player_ratings["Rating"] = 0.0
        
        

    def calculate_ratings(self, step_sizes: list, min_player_match_count: int):

        def apply_ranking_boost(rankings):
            rankings["Rating"] = rankings["Rating"] - rankings["Rating"].loc[MatchRating.avg_player_name] + self.ranking_boost
            return rankings
        
        def remove_excluded_players(rankings):
            excluded_players = self.players[self.players["Excluded"] == True].index
            rankings = rankings.drop(excluded_players, errors="ignore")
            return rankings.drop(columns=["Excluded"])
        
        def add_player_metadata(rankings):
            return rankings.merge(self.players, left_index=True, right_index=True)
        
        def add_match_metadata(rankings):
            
            def get_wins_and_losses(player):
                matches = self.matches[self.matches["IsTieInMatch"] == False]
                wins = matches[self.winner_columns].apply(lambda x: (player in x.values), axis=1).sum()
                losses = matches[self.loser_columns].apply(lambda x: (player in x.values), axis=1).sum()
                return pd.Series([wins, losses], index=["Wins", "Losses"])
            
            rankings[["Wins", "Losses"]] = rankings["ShortName"].apply(get_wins_and_losses)
            return rankings
        
        def filter_min_match_count(rankings):
            return rankings[rankings["Wins"] + rankings["Losses"] >= min_player_match_count]

        def add_opponent_rating(rankings):

            def get_opponent_rating(player):
                matches = self.matches[(self.matches["IsTieInMatch"] == False) & self.matches.apply(lambda x: (player in x.values), axis=1)]
                matches = matches[self.winner_columns + self.loser_columns]

                def get_match_opponent_rating(match):
                    if player in match[self.winner_columns].values:
                        team_columns = self.winner_columns
                        opponent_columns = self.loser_columns
                    else:
                        team_columns = self.loser_columns
                        opponent_columns = self.winner_columns

                    team_players = match[team_columns].dropna().values
                    opponent_players = match[opponent_columns].dropna().values

                    player_team_ranking = rankings.loc[team_players, ["Rating"]].mean().values[0]
                    opponent_team_ranking = rankings.loc[opponent_players, ["Rating"]].mean().values[0]

                    return mean(opponent_team_ranking - player_team_ranking)

                opponent_ratings = matches.apply(get_match_opponent_rating, axis=1)
                return round(opponent_ratings.mean(), 0)
        
            rankings["AVG_Opponent_Rating"] = rankings["ShortName"].apply(get_opponent_rating)
            return rankings
        
        def finalize_rankings(rankings):
            rankings = rankings.sort_values(by="Rating", ascending=False).reset_index(drop=True)
            rankings.index += 1
            rankings["Rating"] = rankings["Rating"].round(0).astype(int)
            return rankings.reset_index()
        

        for step_size in step_sizes:
            rankings = self.__calculate_ratings(step_size)

        rankings["ShortName"] = rankings.index

        
        rankings = apply_ranking_boost(rankings)
        rankings = add_player_metadata(rankings)
        rankings = add_match_metadata(rankings)
        rankings = add_opponent_rating(rankings)
        rankings = remove_excluded_players(rankings)
        rankings = filter_min_match_count(rankings)
        rankings = finalize_rankings(rankings)

        return rankings


        
    
    def __calculate_ratings(self, step_size):
        
        total_match_weights  = self.matches["K"].sum()

        for i in range(MatchRating.max_iterations):
            rating_table = self.__create_rating_table(step_size)
            
            match_score_accuracy = self.__calculate_match_score_accuracy(rating_table)        
            
            player_rating_accuracy = pow(match_score_accuracy.prod(), 1 / total_match_weights)
            accuracy_product = player_rating_accuracy.sort_values(ascending=False)

            best_accuracy = accuracy_product.index[0]
            print(f"Step Size={step_size}, Iteration {i + 1}, Accuracy: {round(accuracy_product.iloc[0], 5)}")

            if best_accuracy == MatchRating.original_rating_name:
                return self.__player_ratings
            
            player, rating_change = best_accuracy            
            self.__player_ratings.loc[player, ["Rating"]] += rating_change
            
        return self.__player_ratings

    def __calculate_match_score_accuracy(self, rating_table: pd.DataFrame):

        match_columns = self.matches.columns.to_list()
        winner_columns = [column for column in match_columns if column.startswith("Winner")]
        loser_columns = [column for column in match_columns if column.startswith("Loser")]

        def apply_match_score_accuracy(match):
            winners = match[winner_columns]
            winners = winners[winners.notna()]
            winner_rating = rating_table.loc[winners].mean()
            
            losers = match[loser_columns]
            losers = losers[losers.notna()]
            loser_rating = rating_table.loc[losers].sum()

            accuracy = pow(winner_rating / (winner_rating + loser_rating), match["K"])

            return accuracy

        return self.matches.apply(apply_match_score_accuracy, axis=1, result_type="expand")

    def __create_rating_table(self, step_size):
        
        ratings = []
        columns = []

        original_rating = self.__player_ratings["Rating"]

        def add_rating(rating, name):
            ratings.append(rating.apply(lambda x: pow(self.rating_scale_base, (x / self.rating_scale_divisor))))
            columns.append(name)

        def add_alternative_rating(player, rating_change):
            alternative_rating = original_rating.copy()
            alternative_rating.loc[player] += rating_change
                
            add_rating(alternative_rating, (player, rating_change))

        add_rating(original_rating, MatchRating.original_rating_name)

        for player in self.players.index:
            add_alternative_rating(player, step_size)
            add_alternative_rating(player, -step_size)
            
        rating_table = pd.concat(ratings, axis=1)
        rating_table.columns = columns

        return rating_table

In [4]:
rating_system = MatchRating(
    # data_sheet_file="Data/Test.xlsx",
    data_sheet_file="Data/UCH_Data.xlsx",
    rating_scale_base=10,
    rating_scale_divisor=400,
    rating_base=1000,
    tie_in_weight=0.5
)

Number of matches loaded:93


In [5]:
rankings = rating_system.calculate_ratings(
    step_sizes=[1024,512,256,128],
    min_player_match_count=0
)
rankings

Step Size=1024, Iteration 1, Accuracy: 0.50766
Step Size=1024, Iteration 2, Accuracy: 0.50766
Step Size=512, Iteration 1, Accuracy: 0.51233
Step Size=512, Iteration 2, Accuracy: 0.5142
Step Size=512, Iteration 3, Accuracy: 0.51608
Step Size=512, Iteration 4, Accuracy: 0.51797
Step Size=512, Iteration 5, Accuracy: 0.51986
Step Size=512, Iteration 6, Accuracy: 0.52176
Step Size=512, Iteration 7, Accuracy: 0.52176
Step Size=256, Iteration 1, Accuracy: 0.52884
Step Size=256, Iteration 2, Accuracy: 0.53636
Step Size=256, Iteration 3, Accuracy: 0.55031
Step Size=256, Iteration 4, Accuracy: 0.56702
Step Size=256, Iteration 5, Accuracy: 0.57085
Step Size=256, Iteration 6, Accuracy: 0.57426
Step Size=256, Iteration 7, Accuracy: 0.57756
Step Size=256, Iteration 8, Accuracy: 0.581
Step Size=256, Iteration 9, Accuracy: 0.58282
Step Size=256, Iteration 10, Accuracy: 0.58464
Step Size=256, Iteration 11, Accuracy: 0.58592
Step Size=256, Iteration 12, Accuracy: 0.58721
Step Size=256, Iteration 13, Acc

,index,Rating,ShortName,Name,Wins,Losses,AVG_Opponent_Rating
0,1,1512,JP,ja_paucia,11,3,-293.0
1,2,1384,RAI,Raito,13,6,-162.0
2,3,1384,MRF,Morfin,15,6,-134.0
3,4,1384,IEY,Gigi,8,2,-205.0
4,5,1256,TNR,TN'ramon,5,4,-43.0
5,6,1256,MAD,MAD MAN,6,5,-81.0
6,7,1128,VB,Variety_Bear,5,4,-28.0
7,8,1128,SZP,szop,6,5,-12.0
8,9,1128,EMB,Ember,2,2,64.0
9,10,1000,KRZ,Kreuzameise,2,2,32.0


In [6]:
rankings = rating_system.calculate_ratings(
    step_sizes=[64,32,16,8],
    min_player_match_count=0
)
rankings

Step Size=64, Iteration 1, Accuracy: 0.61177
Step Size=64, Iteration 2, Accuracy: 0.6123
Step Size=64, Iteration 3, Accuracy: 0.61269
Step Size=64, Iteration 4, Accuracy: 0.61295
Step Size=64, Iteration 5, Accuracy: 0.61304
Step Size=64, Iteration 6, Accuracy: 0.61313
Step Size=64, Iteration 7, Accuracy: 0.61318
Step Size=64, Iteration 8, Accuracy: 0.61322
Step Size=64, Iteration 9, Accuracy: 0.61325
Step Size=64, Iteration 10, Accuracy: 0.61329
Step Size=64, Iteration 11, Accuracy: 0.61332
Step Size=64, Iteration 12, Accuracy: 0.61332
Step Size=32, Iteration 1, Accuracy: 0.61348
Step Size=32, Iteration 2, Accuracy: 0.61358
Step Size=32, Iteration 3, Accuracy: 0.61371
Step Size=32, Iteration 4, Accuracy: 0.61385
Step Size=32, Iteration 5, Accuracy: 0.61392
Step Size=32, Iteration 6, Accuracy: 0.61397
Step Size=32, Iteration 7, Accuracy: 0.61408
Step Size=32, Iteration 8, Accuracy: 0.61412
Step Size=32, Iteration 9, Accuracy: 0.61416
Step Size=32, Iteration 10, Accuracy: 0.61418
Step Si

,index,Rating,ShortName,Name,Wins,Losses,AVG_Opponent_Rating
0,1,1528,JP,ja_paucia,11,3,-262.0
1,2,1480,MRF,Morfin,15,6,-186.0
2,3,1472,IEY,Gigi,8,2,-247.0
3,4,1432,RAI,Raito,13,6,-161.0
4,5,1312,TNR,TN'ramon,5,4,-44.0
5,6,1240,MAD,MAD MAN,6,5,-23.0
6,7,1192,EMB,Ember,2,2,52.0
7,8,1176,VB,Variety_Bear,5,4,-57.0
8,9,1176,SZP,szop,6,5,-20.0
9,10,1064,LEX,LexieUCH,5,8,125.0


In [7]:
rankings = rating_system.calculate_ratings(
    step_sizes=[4,2,1,0.5],
    min_player_match_count=10
)
rankings

Step Size=4, Iteration 1, Accuracy: 0.61453
Step Size=4, Iteration 2, Accuracy: 0.61453
Step Size=4, Iteration 3, Accuracy: 0.61453
Step Size=4, Iteration 4, Accuracy: 0.61453
Step Size=4, Iteration 5, Accuracy: 0.61453
Step Size=4, Iteration 6, Accuracy: 0.61453
Step Size=4, Iteration 7, Accuracy: 0.61453
Step Size=4, Iteration 8, Accuracy: 0.61453
Step Size=4, Iteration 9, Accuracy: 0.61454
Step Size=4, Iteration 10, Accuracy: 0.61454
Step Size=4, Iteration 11, Accuracy: 0.61454
Step Size=4, Iteration 12, Accuracy: 0.61454
Step Size=4, Iteration 13, Accuracy: 0.61454
Step Size=4, Iteration 14, Accuracy: 0.61454
Step Size=4, Iteration 15, Accuracy: 0.61454
Step Size=4, Iteration 16, Accuracy: 0.61454
Step Size=4, Iteration 17, Accuracy: 0.61454
Step Size=4, Iteration 18, Accuracy: 0.61454
Step Size=4, Iteration 19, Accuracy: 0.61454
Step Size=4, Iteration 20, Accuracy: 0.61454
Step Size=4, Iteration 21, Accuracy: 0.61454
Step Size=2, Iteration 1, Accuracy: 0.61454
Step Size=2, Iterati

,index,Rating,ShortName,Name,Wins,Losses,AVG_Opponent_Rating
0,1,1531,JP,ja_paucia,11,3,-263.0
1,2,1484,MRF,Morfin,15,6,-189.0
2,3,1477,IEY,Gigi,8,2,-252.0
3,4,1434,RAI,Raito,13,6,-162.0
4,5,1241,MAD,MAD MAN,6,5,-23.0
5,6,1176,SZP,szop,6,5,-19.0
6,7,1062,LEX,LexieUCH,5,8,128.0
